In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd 
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re
import pickle
import time 
import os

In [3]:
splits = {'test': 'college_mathematics/test-00000-of-00001.parquet', 'validation': 'college_mathematics/validation-00000-of-00001.parquet', 'dev': 'college_mathematics/dev-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/cais/mmlu/" + splits["test"])

In [4]:
def create_prompts_zs(df):
    prompts = []
    answers = []
    for index, row in df.iterrows():
        #
        prompt = "Choose the answer of the given question from the below options.\n"+row['question']+ "\n"+"{1:"+row['choices'][0]+"}" + "\n"+"{2:"+row['choices'][1]+"}" + "\n"+"{3:"+row['choices'][2]+"}"+ "\n"+"{4:"+row['choices'][3]+"}"+"\n" + "answer: {"
        prompts.append(prompt)
        answers.append(str(row['answer']))
    return prompts, answers

In [5]:
prompts_zs, answers = create_prompts_zs(df)

In [6]:
for i in range(len(answers)):
    answers[i] = int(answers[i])

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct", 
    device_map="cuda", 
    torch_dtype="auto", 
    trust_remote_code=True, 
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")

config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

In [9]:
predictions_zeroshot_phi = []
begin = time.time()
for i in range(len(prompts_zs)):
    print(i)
    input_text = prompts_zs[i]
    input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(**input_ids,max_length = 200)
    response = tokenizer.decode(outputs[0])
    #print(response)
    match = re.search(r"answer: \{(\d+)", response)
    extracted_character = match.group(1) if match else '1'
    if(extracted_character in ['1','2','3','4']):
        extracted_number = int(extracted_character)-1
    else:
        extracted_number = 0
    predictions_zeroshot_phi.append(extracted_number)
end = time.time()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [10]:
def calculate_accuracy(predictions, answers):
    valid_pairs = [(pred, ans) for pred, ans in zip(predictions, answers) if pred is not None]
    correct_predictions = sum(1 for pred, ans in valid_pairs if pred == ans)
    accuracy = correct_predictions / len(valid_pairs) if valid_pairs else 0
    return accuracy

In [11]:
filename = 'predictions_zeroshot_phi.pkl'
with open(filename, 'wb') as file:
    pickle.dump(predictions_zeroshot_phi, file)
with open(filename, 'rb') as file:
    predictions_zeroshot_phi = pickle.load(file)

In [13]:
print("Zero shot prompting on Phi")
print("Total inference time:",end-begin,"seconds")
print("Accuracy score:",100*calculate_accuracy(predictions_zeroshot_phi,answers),"%")

Zero shot prompting on Phi
Total inference time: 569.4420506954193 seconds
Accuracy score: 34.0 %
